# Tutorial 01: Running Sumo/Flow Simulations

This tutorial walks a reader through setting up a simulation environment, initializing an attacking model, and subsequently running a simulation to understand the behavior of the simulation.

## Initialize the Simulation: The Ring Road


We will begin with one of the simplest traffic simulation environments, the ringroad. On a ring, a finite number of vehicles following one another. First, we must import the proper network file, in this case `RingNetwork`:

In [1]:
from flow.networks.ring import RingNetwork

## 1. Setting up a Network

This network, as well as all other networks in Flow/Anti-Flow, is parametrized by the following arguments: 
* `name`
* `vehicles`
* `net_params`
* `initial_config`
* `traffic_lights`

The following code shows how to initialize these quantities:

In [2]:
# Initialize parameter : VEHICLES
from flow.core.params import VehicleParams
from flow.controllers.car_following_models import IDMController # A driving model that simulates a human driver.
from flow.controllers.routing_controllers import ContinuousRouter # A Router that keeps vehicles on the ring-road.

vehicles = VehicleParams() # The vehicles object will store different classes of drivers:

# Specify how many human driven vehicles will drive:
num_human_drivers = 20
# Define a driver model human drivers:
vehicles.add("human",
             acceleration_controller=(IDMController, {'noise':0.1}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=num_human_drivers)

In [3]:
# Initialize parameter : NET_PARAMS
from flow.core.params import NetParams

ring_length = 300 # The diameter of the ring-road
net_params = NetParams(additional_params={'length':ring_length,
                                          'lanes':1,
                                          'speed_limit': 30,
                                          'resolution': 40})

In [4]:
# Initialize parameter : INITIAL_CONFIG
from flow.core.params import InitialConfig

initial_config = InitialConfig(spacing="uniform", perturbation=1) # Vehicles start out evenly spaced.

In [5]:
# Initialize parameter : TRAFFIC_LIGHTS
from flow.core.params import TrafficLightParams

traffic_lights = TrafficLightParams() # This is empty, so no traffic lights are used.

## 2. Setting up the Environment

Next, we must set up the Sumo simulation environment. Sumo envrionments in Flow are parametrized by three components:
* `SumoParams`
* `EnvParams`
* `Network`

`SumoParams` specifies simulation-specific variables. These variables maay include the length a simulation step (in seconds), whether to render the GUI when running the experiment, and other variables. Another useful parameter is `emission_path`, which is used to specify the path where the emissions output will be generated. They contain a lot of information about the simulation, for instance the position and speed of each car at each time step. If you do not specify any emission path, the emission file will not be generated.

`EnvParams` specify environment and experiment-specific parameters that either affect the training process or the dynamics of various components within the network. Much like `NetParams`, the attributes associated with this parameter are mostly environment-specific, and can be found in the environment's `ADDITIONAL_ENV_PARAMS` dictionary. Importing the `ADDITIONAL_ENV_PARAMS` variable, we see that it consists of only one entry, "target_velocity", which is used when computing the reward function associated with the environment. We use this default value when generating the `EnvParams` object.

`Network` specifies the network parameters that we initialized in Part 1. These parameters will be passed later as parameters of this simulation.

In [6]:
# Initialize parameter : SUMO_PARAMS
from flow.core.params import SumoParams

#Simulation parameters:
time_step = 0.1 # In seconds, how far each step of the simulation goes.
want_render = True # If we want SUMO to render the environment and display the simulation.
emission_path = 'data' # Result file location.

# Sets the simulation time-step and where data will be recorded.
sim_params = SumoParams(sim_step=time_step, render=want_render, emission_path=emission_path) 

In [7]:
# Initialize parameter : ENV_PARAMS
from flow.core.params import EnvParams
from flow.envs.ring.accel import ADDITIONAL_ENV_PARAMS

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

## 3. Running the Simulation

We now use the initialized parameters above to set up out simulation environemnt:

In [8]:
from flow.envs.ring.accel import AccelEnv
from flow.core.experiment import Experiment

In [9]:
sim_horizon = 3000 # How many simulation steps will be taken -> Runs for 300 seconds

# Compile all simulation parameters as a dictionary.
flow_params = dict(
    exp_tag='ring_no_attack',
    env_name=AccelEnv,
    network=RingNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
    tls=traffic_lights,
)

# Set the number of time steps and initialize the exp object with the simulation parameters.
flow_params['env'].horizon = sim_horizon
exp = Experiment(flow_params)

print('Simulation ready.')

Simulation ready.


Let's now run a simulation which doesn't have any malicious vehicles, and only human drivers:

In [10]:
# Run the simulation and save the data:
sim_res_list = exp.run(1, convert_to_csv=True)

/opt/anaconda3/envs/anti_flow/lib/python3.7/site-packages/numpy-1.18.4-py3.7-macosx-10.7-x86_64.egg/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/envs/anti_flow/lib/python3.7/site-packages/numpy-1.18.4-py3.7-macosx-10.7-x86_64.egg/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Round 0, Return: 2203.0262087596902
data/ring_no_attack_20220706-1635231657143323.98523-0_emission.csv data
Average, std velocities: 7.35862341907247, 0.0
Average, std outflows: 0.0, 0.0
Average, std avg_trip_energy: nan, nan
Average, std avg_trip_time: nan, nan
Average, std total_completed_trips: 0.0, 0.0
Average, std returns: 2203.0262087596902, 0.0
Total time: 20.89385485649109
steps/second: 189.84848215060327


## Visualize simulation results:

First, we need to find the path to the experiment that was just run, which is stored in the exp object. Plotting tools for visualizing the traffic can then be used, which are found in flow\visualize. We plot the spacetime diagram which shows a slight travelling waves develop and then move backwards against the traffic flow.

In [11]:
from importlib import reload
import flow.visualize.visualize_ring as visualize_ring
reload(visualize_ring)

<module 'flow.visualize.visualize_ring' from '/Users/arthursung/Documents/Anti-Flow/flow/visualize/visualize_ring.py'>

In [12]:
import os
import flow.visualize.visualize_ring as visualize_ring

emission_location = os.path.join(os.getcwd(),sim_res_list[1])
visualize_ring.make_ring_spacetime_fig(csv_path = emission_location)

Data loaded.


## Introduce an attacker:

Here we use the compromised model for an ACC which executes a randomized deceleration event (RDA). A single ACC vehicle is defined which has a given attack mangitude and duration. The attacker waits 50 seconds, and to travel 900 m before it executes its attack, and the attack is only executed once.

In [ ]:
import Adversaries.controllers.car_following_adversarial
reload(Adversaries.controllers.car_following_adversarial)
from flow.controllers.car_following_adversarial import ACC_Switched_Controller_Attacked_Single

attack_duration = 10 #How long the vehicle will slow down for.
attack_magnitude = -1.5 #In m/s^2 how strong the braking event is.
adversary_ACC_controller = (ACC_Switched_Controller_Attacked_Single,{
    'warmup_steps':500,
    'distance_threshold_max':800,
    'Total_Attack_Duration':attack_duration,
    'attack_decel_rate':attack_magnitude,
    'display_attack_info':False,
    'V_m':15.0})

# Reinstantiate which vehicles are  being run:
vehicles = VehicleParams()

num_human_drivers = 19
# Same human driver model as previously:
vehicles.add("human",
             acceleration_controller=(IDMController, {'noise':0.1}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=num_human_drivers)

vehicles.add("AV_Adv",
             acceleration_controller=adversary_ACC_controller,
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=1)
             
#initialize the simulation:
flow_params = dict(
    exp_tag='ring_with_attack',
    env_name=AccelEnv,
    network=RingNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
    tls=traffic_lights,
)

# number of time steps
flow_params['env'].horizon = sim_horizon
exp = Experiment(flow_params)
print('Simulation ready.')             

Now we execute the simulation and visualize the results:

In [ ]:
sim_res_list = exp.run(1, convert_to_csv=True)
print('Simulation complete.')

emission_location = os.path.join(os.getcwd(),sim_res_list[1])
visualize_ring.make_ring_spacetime_fig(csv_path = emission_location)

The attack, which happens around position 150, and time 120, first brings traffic to a stand-still and subsequently creates a travelling wave of larger magnitude than the un-attacked traffic.

## Bibliography
[1] Sugiyama, Yuki, et al. "Traffic jams without bottlenecks—experimental evidence for the physical mechanism of the formation of a jam." New journal of physics 10.3 (2008): 033001.

[2] Treiber, Martin, Ansgar Hennecke, and Dirk Helbing. "Congested traffic states in empirical observations and microscopic simulations." Physical review E 62.2 (2000): 1805.